## Reading dataset

In [1]:
import pandas as pd
import numpy as np
import sklearn
import plotly
import pathlib

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
PROJECT_DIR = pathlib.Path().absolute()
DATA_DIR = f'{PROJECT_DIR}/data'
MODEL_DIR = f'{PROJECT_DIR}/models'

RAW_DATA_DIR = f'{DATA_DIR}/raw'
RAW_DATA_IMAGES_DIR = f'{DATA_DIR}/raw/img'

DEV_DATA_PATH = f'{RAW_DATA_DIR}/dev.jsonl'
TRAIN_DATA_PATH = f'{RAW_DATA_DIR}/train.jsonl'
TEST_DATA_PATH = f'{RAW_DATA_DIR}/test.jsonl'

df = pd.read_json(DEV_DATA_PATH, lines=True).set_index('id')

In [4]:
df.head()
df.describe()
df.info()

,img,label,text
id,,,
8291,img/08291.png,1,white people is this a shooting range
46971,img/46971.png,1,bravery at its finest
3745,img/03745.png,1,your order comes to $37.50 and your white priv...
83745,img/83745.png,1,it is time.. to send these parasites back to t...
80243,img/80243.png,1,mississippi wind chime


,label
count,500.000000
mean,0.500000
std,0.500501
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 8291 to 97320
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img     500 non-null    object
 1   label   500 non-null    int64 
 2   text    500 non-null    object
dtypes: int64(1), object(2)
memory usage: 15.6+ KB


In [5]:
(df['label'] == 0).sum()

250

## doc2vec

In [6]:
#!conda install -y gensim 

In [7]:
"""from gensim.models.doc2vec import Doc2Vec


doc2vec = Doc2Vec.load(f'{MODELS_DIR}/doc2vec_apnews_dbow/doc2vec.bin', mmap=None)

print(type(model))
print(type(model.infer_vector(["to", "be", "or", "not"])))
print(model.infer_vector(["to", "be", "or", "not"]).shape)

text_vec = np.array([
    doc2vec.infer_vector(sent.split()) 
    for sent in df['text']
])
text_vec[0]"""

'from gensim.models.doc2vec import Doc2Vec\n\n\ndoc2vec = Doc2Vec.load(f\'{MODELS_DIR}/doc2vec_apnews_dbow/doc2vec.bin\', mmap=None)\n\nprint(type(model))\nprint(type(model.infer_vector(["to", "be", "or", "not"])))\nprint(model.infer_vector(["to", "be", "or", "not"]).shape)\n\ntext_vec = np.array([\n    doc2vec.infer_vector(sent.split()) \n    for sent in df[\'text\']\n])\ntext_vec[0]'

### Catboost

In [8]:
!pip install catboost

In [14]:
df = pd.read_pickle('learning_df.pickle')

In [ ]:
import pickle
pickle.__version__

In [ ]:
!which python

In [35]:
df = pd.read_pickle('learning_df.pickle')
df

,text_vector,image_vector,label,split
id,,,,
42953,"[0.016919259, -0.00021079188, 0.009964041, -0....","[9.243041745321534e-07, 0.0010814660927280784,...",False,train
23058,"[-0.0066814492, -0.001092712, -0.008013967, -0...","[4.7280764192692e-06, 0.0002732583088800311, 5...",False,train
13894,"[0.016481217, 0.0013785894, 0.006280863, -0.00...","[1.44773900956352e-06, 1.3311026123119518e-05,...",False,train
37408,"[0.03513853, -0.0050902003, 0.021591008, -0.00...","[2.800878974085208e-06, 3.871678563882597e-05,...",False,train
82403,"[0.035496537, -0.0066635055, 0.015653217, -0.0...","[1.6485433889101841e-06, 0.0007236988167278469...",False,train
...,...,...,...,...
3869,"[0.0015596877, -0.002792775, -0.0029634463, 0....","[3.028519017789222e-07, 6.190018211782444e-06,...",True,test
23817,"[0.007935097, -0.005352141, 0.008347955, 0.002...","[4.958599575388689e-08, 6.375514089995704e-07,...",True,test
56280,"[0.002048317, 0.0038520799, 0.0025279035, 0.00...","[6.673790630884469e-05, 0.006667210254818201, ...",True,test


In [92]:
learning_df = df

text_vector = np.array([vector for vector in learning_df.text_vector], dtype=np.float32)
image_vector = np.array([vector for vector in learning_df.image_vector], dtype=np.float32)
X = np.concatenate([text_vector, image_vector], axis=-1)
Y = learning_df.label
train_data, dev_data, test_data = X[learning_df.split == 'train'], X[learning_df.split == 'dev'], X[learning_df.split == 'test']
train_labels, dev_labels, test_labels = Y[learning_df.split == 'train'], Y[learning_df.split == 'dev'], Y[learning_df.split == 'test']

train_labels, dev_labels, test_labels = train_labels.astype(int), dev_labels.astype(int), test_labels.astype(int)

In [94]:
train_labels

id
42953    0
23058    0
13894    0
37408    0
82403    0
        ..
10423    1
98203    1
36947    1
16492    1
15937    1
Name: label, Length: 8500, dtype: int64

In [70]:
(train_labels == 0).sum(), len(train_labels)

(5450, 8500)

In [95]:
from sklearn.metrics import roc_auc_score, accuracy_score


def evaluate(model, X, y):
    y_pred_prob = model.predict_proba(X)
    y_pred_prob = y_pred_prob[:, 1]
    y_pred = model.predict(X)
    y = [int(_) for _ in y]
    y_pred = [int(_) for _ in y_pred]
    accur = accuracy_score(y, y_pred)
    roc_auc = roc_auc_score(y, y_pred_prob)
    return accur

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(n_estimators=60, 
                           depth=10,
                           learning_rate=1,
                           #custom_metric=['AUC'],
                           #eval_metric='AUC',
                           verbose=True, 
                           random_seed=42)

model.fit(train_data,
          train_labels,
          verbose=True)

evaluate(model, train_data, train_labels)
evaluate(model, dev_data, dev_labels)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


model = CatBoostClassifier(
    #custom_metric=['AUC'],
    #eval_metric='AUC',
    verbose=False,
    random_seed=42,
)

grid = {
    'n_estimators': (15, 20, 25, 30, 35),
    'learning_rate': (0.1, 0.3, 0.5, 0.7, 0.9),
    'depth': (5, 10, 15)
}
rscv = RandomizedSearchCV(model, grid, random_state=42, verbose=1, cv=3,
                         n_iter=2)
search = rscv.fit(train_data, train_labels)

search.best_params_

search.cv_results_

'train', evaluate(search.best_estimator_, train_data, train_labels)
'valid', evaluate(search.best_estimator_, dev_data, dev_labels)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
